# Importation des bibliothéques

In [1]:
import pandas as pd
import glob

In [2]:
import pandas as pd
import findspark
from pyspark.sql import SparkSession
from pyspark import SparkConf

# for dataframe and udf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *


from pyspark.sql.functions import col
from pyspark.sql import functions as F

from igraph import Graph
from pyspark.sql.window import Window
from pyspark.sql.functions import col

from functools import reduce
from pyspark.sql.functions import collect_set, min as min_, expr,array_min,max
from pyspark.sql.functions import lit

from pyspark.sql.types import StringType, IntegerType, LongType, FloatType, ArrayType,StructType,StructField, BooleanType

import pandas as pd
import glob

import time

import matplotlib.pyplot as plt

from pyspark.sql.types import BooleanType


In [3]:
import os
os.environ["SPARK_HOME"] =  "/root/anaconda3/lib/python3.9/site-packages/pyspark" 
os.environ["JAVA_HOME"] ="/usr"

# Lancer spark

In [4]:
findspark.init() 

def demarrer_spark(NB_CORES):
  local = f"local[{NB_CORES}]"
  # le parametre spark.local.dir indique le repertoire contenant les données temporaires ecrites sur disque lorsque le shufle ne tient pas en memoire
  appName = "PLDAC"
  configLocale = SparkConf().setAppName(appName).setMaster(local).\
  set("spark.executor.memory", "320G").\
  set("spark.driver.memory","320G").\
  set("spark.sql.catalogImplementation","in-memory").\
  set("spark.driver.maxResultSize", "20G").\
  set("spark.local.dir", "/data/bd/spark/tmp").\
  set("spark.sql.execution.arrow.pyspark.enabled", "true")
    

# # to allow sharing in memory arow format between pandas and spark : speeds up the creation of a spark df from a pandas df
# spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")


  spark = SparkSession.builder.config(conf = configLocale).getOrCreate()
  sc = spark.sparkContext
  sc.setLogLevel("ERROR")
  
  # spark.conf.set("spark.sql.autoBroadcastJoinThreshold","-1")
    
  # On ajuste l'environnement d'exécution des requêtes à la taille du cluster (NB_CORES coeurs)
  shuffle_partitions = 3 * NB_CORES
  print("shuffle", shuffle_partitions)
  spark.conf.set("spark.sql.shuffle.partitions", str(shuffle_partitions))    

  print("session démarrée, son id est ", sc.applicationId)
  return spark


# Lecture des données

In [10]:
spark = demarrer_spark(60)


shuffle 180
session démarrée, son id est  local-1714395275570


In [11]:
parquet_folder = "/data/bd/dataset/proteine/80_80/G99/graph_deduplicated_v2/"

df = spark.read.parquet(parquet_folder)
df.show(20)

+-----------+------------+-----------+-------------+------------+----------------+-------+------------+---+
|   query_id|query_length|  target_id|target_length|match_length|percent_identity|e_value|relative_sim|  r|
+-----------+------------+-----------+-------------+------------+----------------+-------+------------+---+
|100003172:0|          89|143094166:3|          799|          89|           100.0|2.2E-46|         1.0|  1|
|100003668:0|         102| 48815973:0|          222|         102|            99.0|3.9E-55|        0.99|  1|
|100004301:0|         169|106189136:2|          695|         169|            99.4|3.9E-94|       0.994|  1|
|100004301:0|         169|117840121:3|          264|         169|           100.0|1.9E-94|         1.0|  1|
|100004301:0|         169| 44038038:1|          524|         169|           100.0|5.4E-94|         1.0|  1|
|100004301:0|         169| 49847952:5|          458|         169|            99.4|1.2E-93|       0.994|  1|
|100004301:0|         169| 6

In [12]:
df = df[['target_id', 'query_id']]
df.count()

341807741

In [16]:
# Répartissez le DataFrame en n partitions
nb_partitions = [60]
for num_partitions in nb_partitions :
    df_repartitioned = df.repartition(num_partitions, 'target_id')

    # Écrivez le DataFrame réparti sur le disque
    # Vous pouvez ajuster le format et l'emplacement de sortie selon vos besoins
    output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions"
    df_repartitioned.write.mode("overwrite").parquet(output_path)


In [17]:
import os

# Chemin vers le dossier contenant les fichiers à renommer
output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions"

# Liste de tous les fichiers non cachés dans le dossier
files = [file for file in os.listdir(output_path) if not (file.startswith('.') or file=="_SUCCESS")]  # Exclure les fichiers cachés
print(len(files))

# Numéro de départ pour le renommage
start_index = 0

# Parcourir tous les fichiers
for filename in files:
    # Construction du nouveau nom de fichier en utilisant le numéro actuel
    new_filename = f"partition_{start_index}.parquet"
    
    # Chemin complet vers l'ancien et le nouveau fichier
    old_file_path = os.path.join(output_path, filename)
    new_file_path = os.path.join(output_path, new_filename)
    
    # Renommer le fichier
    os.rename(old_file_path, new_file_path)
    
    # Affichage pour confirmation
    print(f"Renommage de {filename} en {new_filename}")
    
    # Incrémenter le numéro de départ pour le prochain fichier
    start_index += 1


60
Renommage de part-00007-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_0.parquet
Renommage de part-00056-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_1.parquet
Renommage de part-00046-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_2.parquet
Renommage de part-00025-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_3.parquet
Renommage de part-00043-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_4.parquet
Renommage de part-00001-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_5.parquet
Renommage de part-00004-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_6.parquet
Renommage de part-00033-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_7.parquet
Renommage de part-00034-f85b16f2-f208-4847-9714-529407e04214-c000.snappy.parquet en partition_8.parquet
Renommage de part-00010-f85b16f2-f208-4847-9714-529407e04214-

In [16]:
num_partitions = 50
output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions/part-00049-bbae4cde-d0d9-4958-9a0a-5953aa180d0c-c000.snappy.parquet"

df = spark.read.parquet(output_path)
num_rows = df.count()
print(f"Le fichier {output_path} contient {num_rows} lignes.")


Le fichier /data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/50_partitions/part-00049-bbae4cde-d0d9-4958-9a0a-5953aa180d0c-c000.snappy.parquet contient 6904102 lignes.


In [6]:
num_partitions = 150
output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions/part-00149-cf23f7e6-df01-44ab-a44f-c9faa14d4ab4-c000.snappy.parquet"

df = spark.read.parquet(output_path)
num_rows = df.count()
print(f"Le fichier {output_path} contient {num_rows} lignes.")


Le fichier /data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/150_partitions/part-00149-cf23f7e6-df01-44ab-a44f-c9faa14d4ab4-c000.snappy.parquet contient 2305246 lignes.


In [11]:
nb_partitions = [500]
for num_partitions in nb_partitions:
    output_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/{num_partitions}_partitions/"

    with os.scandir(output_path) as entries:
        i=0
        for entry in entries:
            if entry.is_file():
                filename = entry.name
                if filename.endswith(".parquet"):
                    print(filename)
                    print(os.path.join(output_path, f"partition_{i}.parquet"))
                    print(os.path.join(output_path, filename))
                    os.rename(os.path.join(output_path, filename), os.path.join(output_path, f"partition_{i}.parquet"))
                    print(i)
                    i+=1


part-00389-bc8881d6-2581-4088-9a36-6035a1a4ba6d-c000.snappy.parquet
/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/500_partitions/partition_0.parquet
/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/500_partitions/part-00389-bc8881d6-2581-4088-9a36-6035a1a4ba6d-c000.snappy.parquet
0
part-00415-bc8881d6-2581-4088-9a36-6035a1a4ba6d-c000.snappy.parquet
/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/500_partitions/partition_1.parquet
/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/500_partitions/part-00415-bc8881d6-2581-4088-9a36-6035a1a4ba6d-c000.snappy.parquet
1
part-00448-bc8881d6-2581-4088-9a36-6035a1a4ba6d-c000.snappy.parquet
/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/500_partitions/partition_2.parquet
/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/partitions/500_partitions/part-00448-bc8881d6-2581-4088-9a36-6035a1a4b

# Partionnement des arêtes en plusieurs sous ensembles

In [8]:
# # Nombre de partitions
# num_partitions = 200

# # Ajout d'une nouvelle colonne 'partition' basée sur le modulo de seqID1
# df['partition'] = df['target_id'].apply(lambda x: hash(x) % num_partitions)

# # Afficher le DataFrame partitionné
# df

,target_id,query_id,partition
0,EukProt-v2_GFUD01036155.1.p1,100000481:3,111
1,12062594:2,100003668:0,187
2,56659524:1,100003668:0,163
3,136300595:3,100004174:2,142
4,142753793:3,100004301:0,153
...,...,...,...
341807736,METdb_00414-1-Transcript-29652.p1,TRINITY-DN9947-c4-g3-i1.p1,174
341807737,EukProt-v2_CAMPEP_0184944524,TRINITY-DN9969-c0-g4-i1.p1,70
341807738,METdb_00414-1-Transcript-29735.p1,TRINITY-DN9976-c0-g1-i3.p1,60
341807739,141777669:5,TRINITY-DN9989-c4-g4-i1.p1,141


In [9]:
partition_freq = df.groupby('partition').size().reset_index(name='count')
partitions = partition_freq['partition'].unique()
for partition in partitions:
    df_partition = df[df['partition'] == partition]
    file_path = f"/data/bd/dataset/proteine/80_80/G99/TEST_RACHA_CALCUL_COMPOSANTES/200_partitions/partition_{partition}.parquet"
    df_partition.to_parquet(file_path, index=False, engine='pyarrow')

In [9]:
partition_freq = df.groupby('partition').sizecount(à
partition_freq

AnalysisException: cannot resolve '`partition`' given input columns: [query_id, target_id];
'Aggregate ['partition], ['partition, count(1) AS count#77L]
+- Project [target_id#2, query_id#0]
   +- Relation[query_id#0,query_length#1L,target_id#2,target_length#3L,match_length#4L,percent_identity#5,e_value#6,relative_sim#7,r#8] parquet


In [34]:
spark.stop()